<a href="https://colab.research.google.com/github/crazyboyonline/-/blob/main/%E6%8E%A8%E8%8D%90%E7%B3%BB%E7%BB%9F%E7%9A%84%E4%B8%AA%E6%80%A7%E5%8C%96%E6%8E%92%E5%90%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

在之前只考虑了明确的反馈，并根据观察到的评级对模型进行了训练和测试。这种方法有两个缺点：首先，大多数反馈在现实世界的场景中不是显式的而是隐式的，而且显式反馈的收集成本可能更高。其次，可以预测用户兴趣的未观察到的用户-项目对被完全忽略，这使得这些方法不适用于评级不是随机丢失而是因为用户偏好的情况。未观察到的用户-项目对是真实负反馈（用户对项目不感兴趣）和缺失值（用户将来可能与项目交互）的混合体。我们只是忽略了矩阵分解和 AutoRec 中的未观察到的对。清楚地，

为此，一类旨在从隐式反馈中生成排名推荐列表的推荐模型得到了普及。一般来说，个性化排名模型可以通过逐点、成对或逐列表方法进行优化。逐点方法一次考虑单个交互，并训练分类器或回归器来预测个人偏好。矩阵分解和 AutoRec 使用逐点目标进行优化。成对方法为每个用户考虑一对项目，并旨在近似该对的最佳排序。通常，成对方法更适合排序任务，因为预测相对顺序让人联想到排序的本质。Listwise 方法近似于整个项目列表的排序，例如，NDCG）。但是，listwise 方法比逐点或成对方法更复杂和计算密集。在本节中，我们将介绍两个成对的目标/损失，贝叶斯个性化排名损失和铰链损失，以及它们各自的实现。

In [ ]:
!pip install d2l==0.14
!pip install mxnet==1.7.0.post1

     |████████████████████████████████| 48 kB 2.6 MB/s 
     |████████████████████████████████| 55.0 MB 130 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


我们将实现基类mxnet.gluon.loss.Loss并重写该forward方法来构造贝叶斯个性化排名损失。我们首先导入 Loss 类和 np 模块。

In [ ]:
from mxnet import gluon,np,npx
npx.set_np()

BPR loss的实现如下

In [ ]:
class BPRLoss(gluon.loss.Loss):
  def __init__(self,weight = None, batch_axis = 0,**kwargs):
    super(BPRLoss,self).__init__(weight = None, batch_axis= 0,**kwargs)

  def forward(self,positive,negative):
    distances = positive - negative
    loss = - np.sum(np.log(npx.sigmoid(distances)),0,keepdims =True)
    return loss

Hinge Loss and its Implementation
m 是安全边际大小。它旨在将消极项目推离积极项目。与 BPR 类似，它旨在优化正负样本之间的相关距离，而不是绝对输出，使其非常适合推荐系统。

In [ ]:
class HingeLossbRec(gluon.loss.Loss):
  def __init__(self, weight = None, batch_axis = 0, **kwargs):
      super(HingeLossbRec,self).__init__(weight = None, batch_axis = 0, **kwargs)
  
  def forward(self,positive,negative,margin = 1):
    distances = positive - negative
    loss = np.sum(np.maximum(- distances + margin,0))